In [5]:
import sys
import copy

#global veriables

visited_nodes=0

def count_visited():
    global visited_nodes
    visited_nodes+=1 

In [6]:
class Node():
    
    def __init__(self ,status , maximizing, player, parent=None):
        self.children = list()
        self.score = 0
        self.maximizing = maximizing
        self.player = player
        self.best_child = None
        
        if parent is not None:
            parent.children.append(self)
        
    def __hash__(self):
        
        return hash(str(self.status))

In [20]:
class environment :
    AI = 'O'
    HUMAN = 'X'
    TIE = 'tie'
    values = {'X':1,'O':-1,'tie':0}
    maximizing_player = HUMAN
     
    def __init__(self,board,human):
        self.game_Board=board
        self.HUMAN = human
        
        if(human == 'X'):
            self.AI = 'O'
        else:
            self.AI = 'X'
        
    def if_game_over(self,game_board):

        v = None
        #Diagonal self.values check
        if(game_board[0][0] == game_board[1][1] == game_board[2][2] == self.HUMAN):
            v = self.HUMAN
        elif(game_board[0][0] == game_board[1][1] == game_board[2][2] == self.AI):
            v = self.AI

        if(game_board[0][2] == game_board[1][1] == game_board[2][0] == self.HUMAN):
            v = self.HUMAN
        elif(game_board[0][2] == game_board[1][1] == game_board[2][0] == self.AI):
            v = self.AI

        #Check horizontally
        for i in range(0,3):
            if(game_board[i][0] == game_board[i][1] == game_board[i][2] == self.HUMAN):
                v = self.HUMAN
            elif(game_board[i][0] == game_board[i][1] == game_board[i][2] == self.AI):
                v = self.AI

        #Check vertically
        for j in range(0,3):
            if(game_board[0][j] == game_board[1][j] == game_board[2][j] == self.HUMAN):
                v = self.HUMAN
            elif(game_board[0][j] == game_board[1][j] == game_board[2][j] == self.AI):
                v = self.AI

        if(v != None):
            return self.values[v]

        #check for tie
        t = True
        for i in range(0,3):
            for j in range(0,3):
                if game_board[i][j]=='-' and v == None:
                    t=False
        
        if(t):
            v = self.values[self.TIE]
        
        return v
   
    def human_move(self):
        index = int(input("your move between 0-8:"))  #0-8
        i = int(index/3)
        j = int(index%3)
        if(self.game_Board[i][j] == '-'):
            self.game_Board[i][j] = self.HUMAN
        else:
            self.human_move()
            
    def print_board(self):
      for i in self.game_Board:
            print(*i," ",sep = ' ')
        
    def play(self,a):
        global visited_nodes
        self.a=a
        turn = True
        self.print_board()
        while (self.if_game_over(self.game_Board)==None):
          if turn:
            print("AI turn:")
            self.game_Board = self.a.best_move(self.game_Board)
            print(visited_nodes)
            visited_nodes=0
          else:
            print("Human turn:")
            self.human_move()
          turn = not(turn)
          self.print_board()

    def posMoves(self,player,game_board):
      moves = []
      for i in range(0,3):
            for j in range(0,3):
                if game_board[i][j]=='-':
                  new = copy.deepcopy(game_board)
                  new[i][j]=player
                  # print(game_board)
                  moves.append(new)
      # print(moves)
      return moves


In [23]:
class agent:
    
    def __init__(self,env):
        self.env = env
        self.maximizing = self.env.maximizing_player
        self.AI = self.env.AI
        self.HUMAN = self.env.HUMAN
        
    def minmax(self,game_board,node):
        count_visited()
        env = self.env

        node.status = env.if_game_over(game_board)
        
        if(node.status != None):
            return node

        moves=env.posMoves(node.player,game_board)  

        if(node.maximizing):
          bestScore = int(-2)
          # moves=env.posMoves(turn,game_board)

          for m in moves:
            child = Node(node.status,False,self.AI,node)
            child = self.minmax(m,child)
            node.children.append(child)
            bestScore = max(bestScore,child.status)
          node.status = bestScore
          return node

        else:
          bestScore = int(2)
          
          for m in moves:
            child = Node(node.status,True,self.HUMAN,node)
            child = self.minmax(m,child)
            node.children.append(child)
            bestScore = min(bestScore,child.status)
          node.status = bestScore
          return node

    def best_move(self,gb):
        bestScore = 2
        move = []
        moves = self.env.posMoves(self.AI,gb)
        for m in moves:
          node = Node(None,True,self.HUMAN,None)
          node = self.minmax(m,node)
          print(m," ",bestScore," ",node.status)
          if(node.status<bestScore):
            print(m)
            bestScore=node.status
            move=m

        # print(move)
        return move 

In [24]:

init_game_board = [['-','-','-'],['-','-','-'],['-','-','-']]

env = environment(init_game_board,'X')

a = agent(env)

env.play(a)
# print(env.if_game_over(init_game_board))
# a.minmax(init_game_board,False,env.AI)

print(visited_nodes)

- - -  
- - -  
- - -  
AI turn:
[['O', '-', '-'], ['-', '-', '-'], ['-', '-', '-']]   2   0
[['O', '-', '-'], ['-', '-', '-'], ['-', '-', '-']]
[['-', 'O', '-'], ['-', '-', '-'], ['-', '-', '-']]   0   0
[['-', '-', 'O'], ['-', '-', '-'], ['-', '-', '-']]   0   0
[['-', '-', '-'], ['O', '-', '-'], ['-', '-', '-']]   0   0
[['-', '-', '-'], ['-', 'O', '-'], ['-', '-', '-']]   0   0
[['-', '-', '-'], ['-', '-', 'O'], ['-', '-', '-']]   0   0
[['-', '-', '-'], ['-', '-', '-'], ['O', '-', '-']]   0   0
[['-', '-', '-'], ['-', '-', '-'], ['-', 'O', '-']]   0   0
[['-', '-', '-'], ['-', '-', '-'], ['-', '-', 'O']]   0   0
549945
O - -  
- - -  
- - -  
Human turn:
your move between 0-8:8
O - -  
- - -  
- - X  
AI turn:
[['O', 'O', '-'], ['-', '-', '-'], ['-', '-', 'X']]   2   1
[['O', 'O', '-'], ['-', '-', '-'], ['-', '-', 'X']]
[['O', '-', 'O'], ['-', '-', '-'], ['-', '-', 'X']]   1   -1
[['O', '-', 'O'], ['-', '-', '-'], ['-', '-', 'X']]
[['O', '-', '-'], ['O', '-', '-'], ['-', '-', 'X']

In [ ]:
import numpy as np
import time

In [ ]:
class Node():
    
    def __init__(self, state, depth, maximiser, player, parent=None,):
        self.state = state
        self.children = list()
        self.score = 0
        self.depth = depth
        self.maximiser = maximiser
        self.player = player
        self.best_child = None
        
        if parent is not None:
            parent.children.append(self)
        
    def __hash__(self):
        
        return hash(str(self.state))

In [ ]:
class Environment():
    
    def __init__(self, start_state=None):
        
        if start_state is None:
            self.start_state = np.array([['.','.','.'],['.','.','.'],['.','.','.']])
        else:
            self.start_state = start_state
    def get_moves(self, state, player):
        
        new_states = []
        spaces = []
        for i in range(3):
            for j in range(3):
                if state[i][j]=='.':
                    new_state = state.copy()
                    new_state[i,j] = player
                    new_states.append(new_state)
        
        return new_states
    
    def check_terminal(self, state):
        for i in range(3):
            for j in range(3):
                if state[i][j]=='.':
                    return False
        
        return True
    
    def evaluate(self, state):
        
        for val in range(3):
            if state[val,0] == state[val,1] == state[val,2]!='.':
                if state[val, 0]=='x':
                    return 1
                else:
                    return -1
            
            if state[0,val] == state[1,val] == state[2,val]!='.':
                if state[0,val]=='x':
                    return 1
                else:
                    return -1
        
        if state[0,0] == state[1,1] == state[2,2]!='.':
            if state[0,0]=='x':
                return 1
            else:
                return -1
        
        if state[0,2] == state[1,1] == state[2,0]!='.':
            if state[0,2]=='x':
                return 1
            else:
                return -1
        
        return 0

    def get_start_state(self):
        return self.start_state

Alpha beta agent:

In [ ]:
class AlphaBetaAgent():
    
    def __init__(self, env, maximiser):
        
        self.env = env
        self.start_state = env.get_start_state()
        self.root_node = None
        self.neginf = -10**18
        self.posinf = 10**18
        self.maximiser = maximiser
        self.player = 'x' if maximiser else 'o';
        self.explored_nodes = 0
    
    
    def minimax(self, node, alpha, beta):
        
        self.explored_nodes+=1
        score = self.env.evaluate(node.state)
        if score!=0:
            node.score = score
            return node
        
        if self.env.check_terminal(node.state):
            node.score = 0
            return node
        
        if node.maximiser:
            
            best_score = self.neginf
            best_depth = self.posinf
            next_moves = self.env.get_moves(node.state, node.player)
            for move in next_moves:
                child = Node(state = move, depth=node.depth+1, 
                             maximiser=not node.maximiser, player='o', parent=node)
                
                child= self.minimax(node = child, alpha=alpha, beta=beta)
                node.children.append(child)
                
                if best_score<child.score or (best_score==child.score and child.depth<best_depth):
                    best_score = child.score
                    best_depth = child.depth
                    node.best_child = child
                
                alpha = max(alpha, best_score)
                if beta <= alpha:
                    break
            
            node.depth = best_depth
            node.score = best_score
            
            return node
        
        else:
            best_score = self.posinf
            best_depth = self.posinf
            next_moves = self.env.get_moves(node.state, node.player)
            
            for move in next_moves:
                child = Node(state = move, depth=node.depth+1, 
                             maximiser=not node.maximiser, player='x', parent=node)
                
                child = self.minimax(node = child, alpha=alpha, beta=beta)
                node.children.append(child)
                
                if best_score>child.score  or (best_score==child.score and child.depth<best_depth):
                    best_score = child.score
                    best_depth = child.depth
                    node.best_child = child
                
                beta = min(beta, best_score)
                if beta<=alpha:
                    break
            
            node.depth = best_depth
            node.score = best_score
            
            return node

    def run(self):
        
        self.root_node = Node(state=self.start_state, depth=0, maximiser=self.maximiser,
                             player=self.player, parent=None)
        
        self.root_node = self.minimax(node = self.root_node, alpha = self.neginf, beta = self.posinf)
        
    def print_nodes(self):
        
        node = self.root_node
        
        while node is not None:
            print(node.state)
            node = node.best_child


start_state = np.array([['.','.','.'],['.','.','.'],['.','.','.']])
env = Environment(start_state = start_state)
agent = AlphaBetaAgent(env, maximiser=True)
agent.run()
agent.print_nodes()


In [22]:
# class AlphaBetaAgent():

#     def __init__(self,env):
#         self.env = env
#         self.maximizing = self.env.maximizing_player
#         self.AI = self.env.AI
#         self.HUMAN = self.env.HUMAN
    
#     # def __init__(self, env, maximiser):
        
#     #     self.env = env
#     #     self.start_state = env.get_start_state()
#     #     self.root_node = None
#     #     self.neginf = -10**18
#     #     self.posinf = 10**18
#     #     self.maximiser = maximiser
#     #     self.player = 'x' if maximiser else 'o';
#     #     self.explored_nodes = 0
    
    
#     def minimax(self, bg , node, alpha, beta):
#         count_visited()
#         env = self.env

#         node.status = env.if_game_over(game_board)
        
#         if(node.status != None):
#             return node
        
#         if node.maximizing:

#             best_score = self.neginf
#             next_moves = self.env.posMoves(node.player,gb)

#             for move in next_moves:
#                 child = Node(node.status,False,self.AI,node)
                
#                 child= self.minimax(bg ,node = child, alpha=alpha, beta=beta)
#                 node.children.append(child)
                
#                 if best_score<child.score or (best_score==child.score):
#                     best_score = child.score
#                     node.best_child = child
                
#                 alpha = max(alpha, best_score)
#                 if beta <= alpha:
#                     break

#             node.score = best_score
            
#             return node
        
#         else:
#             best_score = self.posinf
#             best_depth = self.posinf
#             next_moves = self.env.get_moves(node.state, node.player)
            
#             for move in next_moves:
#                 child = Node(node.status,True,self.HUMAN,node)
                
#                 child = self.minimax(bg, node = child, alpha=alpha, beta=beta)
#                 node.children.append(child)
                
#                 if best_score>child.score  or (best_score==child.score):
#                     best_score = child.score
#                     node.best_child = child
                
#                 beta = min(beta, best_score)
#                 if beta<=alpha:
#                     break

#             node.score = best_score
            
#             return node

#     def run(self):
        
#         self.root_node = Node(state=self.start_state, depth=0, maximiser=self.maximiser,
#                              player=self.player, parent=None)
        
#         self.root_node = self.minimax(node = self.root_node, alpha = self.neginf, beta = self.posinf)
        
#     def print_nodes(self):
        
#         node = self.root_node
        
#         while node is not None:
#             print(node.state)
#             node = node.best_child